Ensure that all necessary Python libraries (listed below) are downloaded

In [73]:
import numpy as np
import pandas as pd
import os
import datetime
import xml.etree.ElementTree as ET
import holidays
import tkinter.filedialog

In [74]:
root = tkinter.Tk()
request_file = tkinter.filedialog.askopenfilename(initialdir=os.getcwd(),filetypes=[('XML Files', '*.xml')])
root.destroy()
xml_file_name = request_file.split('/')[-1]
xml_file_name

'mry_tides_annual2021.xml'

In [75]:
annual_tides = ET.parse(xml_file_name).getroot()

In [79]:
annual_tides[4].text

'Monterey'

In [78]:
annual_tides[5].text

'CA'

In [66]:
station = annual_tides[4].text
station

'Monterey'

In [67]:
tides_year = annual_tides[8].text[:4]
tides_year

'2021'

In [68]:
col_names = list()
for n in range(0,6):
    col_names.append(annual_tides[-1][0][n].tag)
col_names.remove('date')
col_names.remove('time')
col_names.insert(0,'date_time')
col_names

['date_time', 'day', 'pred_in_ft', 'pred_in_cm', 'highlow']

In [69]:
f = annual_tides.find(annual_tides[-1].tag)
df_tides = pd.DataFrame(index=range(0,len(list(f.iter(annual_tides[-1][0].tag)))),columns=col_names)
df_tides

,date_time,day,pred_in_ft,pred_in_cm,highlow
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1406,NaN,NaN,NaN,NaN,NaN
1407,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN
1409,NaN,NaN,NaN,NaN,NaN


In [70]:
for i in df_tides.index:
    df_tides['date_time'].iloc[i]=datetime.datetime.strptime(annual_tides[-1][i][0].text+annual_tides[-1][i][2].text,'%Y/%m/%d%I:%M %p')
    df_tides[annual_tides[-1][0][1].tag].iloc[i]=annual_tides[-1][i][1].text
    df_tides[annual_tides[-1][0][3].tag].iloc[i]=float(annual_tides[-1][i][3].text)
    df_tides[annual_tides[-1][0][4].tag].iloc[i]=float(annual_tides[-1][i][4].text)
    df_tides[annual_tides[-1][0][5].tag].iloc[i]=annual_tides[-1][i][5].text
df_tides

,date_time,day,pred_in_ft,pred_in_cm,highlow
0,2021-01-01 01:00:00,Fri,3.85,117.0,H
1,2021-01-01 04:55:00,Fri,3.04,93.0,L
2,2021-01-01 10:52:00,Fri,5.82,177.0,H
3,2021-01-01 18:31:00,Fri,-0.82,-25.0,L
4,2021-01-02 01:40:00,Sat,3.94,120.0,H
...,...,...,...,...,...
1406,2021-12-30 20:42:00,Thu,3.62,110.0,H
1407,2021-12-31 01:01:00,Fri,2.36,72.0,L
1408,2021-12-31 07:30:00,Fri,6.64,202.0,H
1409,2021-12-31 15:01:00,Fri,-1.16,-35.0,L


In [71]:
df_copy = df_tides.copy()
for i in df_copy.index:
    if not df_copy['highlow'][i] == 'L':
        df_copy.drop(index=i,inplace=True)
    else:
        if df_copy['day'][i] == 'Sat' or df_copy['day'][i] =='Sun' or df_copy['date_time'][i].date() in holidays.US(years=df_copy['date_time'][i].date().year):
            if not 8 <= df_copy['date_time'][i].hour <= 20:
                df_copy.drop(index=i,inplace=True)
            else: pass
        else:
            if not 16 <= df_copy['date_time'][i].hour <= 20:
                df_copy.drop(index=i,inplace=True)
            else: pass
for i in df_copy.index:
    if df_copy['pred_in_cm'][i]>=df_copy['pred_in_cm'].mean():
        df_copy.drop(index=i,inplace=True)
    else: pass
df_copy

,date_time,day,pred_in_ft,pred_in_cm,highlow
3,2021-01-01 18:31:00,Fri,-0.82,-25.0,L
7,2021-01-02 19:10:00,Sat,-0.63,-19.0,L
11,2021-01-03 19:51:00,Sun,-0.3,-9.0,L
15,2021-01-04 20:34:00,Mon,0.15,5.0,L
33,2021-01-09 14:04:00,Sat,-0.58,-18.0,L
...,...,...,...,...,...
1363,2021-12-19 17:11:00,Sun,-0.53,-16.0,L
1367,2021-12-20 17:47:00,Mon,-0.53,-16.0,L
1371,2021-12-21 18:23:00,Tue,-0.47,-14.0,L
1375,2021-12-22 19:01:00,Wed,-0.35,-11.0,L


In [72]:
df_goog_cal = pd.DataFrame(columns=['Subject','Start Date','Start Time','End Date','End Time','All Day Event','Private','Description','Location'],index=df_copy.index)
for i in df_goog_cal.index:
    df_goog_cal['Subject'].loc[i] = 'Low Tide'
    df_goog_cal['Start Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['Start Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['End Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['End Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['All Day Event'].loc[i] = False
    df_goog_cal['Private'].loc[i] = False
    df_goog_cal['Description'].loc[i] = 'Below Average Low Tide'
    df_goog_cal['Location'].loc[i] = f'{annual_tides[4].text},{annual_tides[5].text}'
df_goog_cal

,Subject,Start Date,Start Time,End Date,End Time,All Day Event,Private,Description,Location
3,Low Tide,2021-01-01,18:31:00,2021-01-01,18:31:00,False,False,Below Average Low Tide,"Monterey, CA"
7,Low Tide,2021-01-02,19:10:00,2021-01-02,19:10:00,False,False,Below Average Low Tide,"Monterey, CA"
11,Low Tide,2021-01-03,19:51:00,2021-01-03,19:51:00,False,False,Below Average Low Tide,"Monterey, CA"
15,Low Tide,2021-01-04,20:34:00,2021-01-04,20:34:00,False,False,Below Average Low Tide,"Monterey, CA"
33,Low Tide,2021-01-09,14:04:00,2021-01-09,14:04:00,False,False,Below Average Low Tide,"Monterey, CA"
...,...,...,...,...,...,...,...,...,...
1363,Low Tide,2021-12-19,17:11:00,2021-12-19,17:11:00,False,False,Below Average Low Tide,"Monterey, CA"
1367,Low Tide,2021-12-20,17:47:00,2021-12-20,17:47:00,False,False,Below Average Low Tide,"Monterey, CA"
1371,Low Tide,2021-12-21,18:23:00,2021-12-21,18:23:00,False,False,Below Average Low Tide,"Monterey, CA"
1375,Low Tide,2021-12-22,19:01:00,2021-12-22,19:01:00,False,False,Below Average Low Tide,"Monterey, CA"


In [10]:
df_goog_cal.to_csv(f'{station}_lowest_tides{tides_year}.csv')